# Mohr's Circle using Hibbler Sign Convension

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider


def plot_strain_and_mohr_circle(angle, epsilon_x, epsilon_y, gamma_xy):
  # Convert input from microstrain to actual strain
  epsilon_x = epsilon_x / 1e6
  epsilon_y = epsilon_y / 1e6
  gamma_xy = gamma_xy / 1e6

  # Define square vertices
  square = np.array([
    [-0.5, -0.5],
    [0.5, -0.5],
    [0.5, 0.5],
    [-0.5, 0.5],
    [-0.5, -0.5],  # Close the square
  ])

  # Rotation matrix
  theta = np.radians(angle)
  rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                              [np.sin(theta), np.cos(theta)]])

  # Rotate square
  rotated_square = square @ rotation_matrix.T

  # Strain transformation equations
  epsilon_x_prime = (epsilon_x * np.cos(theta)**2 +
                     epsilon_y * np.sin(theta)**2 +
                     gamma_xy * np.sin(theta) * np.cos(theta))
  epsilon_y_prime = (epsilon_x * np.sin(theta)**2 +
                     epsilon_y * np.cos(theta)**2 -
                     gamma_xy * np.sin(theta) * np.cos(theta))
  gamma_xy_prime = 2 * (epsilon_y - epsilon_x) * np.sin(theta) * np.cos(
    theta) + gamma_xy * (np.cos(theta)**2 - np.sin(theta)**2)

  # Right face midpoint
  right_midpoint = np.array([0.5, 0])
  right_midpoint_rotated = right_midpoint @ rotation_matrix.T

  # Top face midpoint
  top_midpoint = np.array([0, 0.5])
  top_midpoint_rotated = top_midpoint @ rotation_matrix.T

  # Normalize vectors for fixed length
  right_normal_vector = np.array([1, 0]) * (1 if epsilon_x_prime >= 0 else -1)
  right_shear_vector = np.array([0, 1]) * (1 if gamma_xy_prime >= 0 else -1)

  top_normal_vector = np.array([0, 1]) * (1 if epsilon_y_prime >= 0 else -1)
  top_shear_vector = np.array([1, 0]) * (1 if gamma_xy_prime >= 0 else -1)

  # Rotate vectors
  right_normal_vector_rotated = right_normal_vector @ rotation_matrix.T
  right_shear_vector_rotated = right_shear_vector @ rotation_matrix.T

  top_normal_vector_rotated = top_normal_vector @ rotation_matrix.T
  top_shear_vector_rotated = top_shear_vector @ rotation_matrix.T

  # Normal strain arrows
  right_normal_start = right_midpoint_rotated if epsilon_x_prime >= 0 else (
    right_midpoint_rotated - right_normal_vector_rotated)
  right_normal_end = right_normal_start + right_normal_vector_rotated

  top_normal_start = top_midpoint_rotated if epsilon_y_prime >= 0 else (
    top_midpoint_rotated - top_normal_vector_rotated)
  top_normal_end = top_normal_start + top_normal_vector_rotated

  # Shear strain arrows
  right_shear_center = (right_normal_start + right_normal_end) / 3
  right_shear_start = (right_shear_center - 0.5 * right_shear_vector_rotated)
  right_shear_end = right_shear_start + right_shear_vector_rotated

  top_shear_center = (top_normal_start + top_normal_end) / 3
  top_shear_start = (top_shear_center - 0.5 * top_shear_vector_rotated)
  top_shear_end = top_shear_start + top_shear_vector_rotated

  # Mohr Circle calculations for strain
  center = (epsilon_x + epsilon_y) / 2
  radius = np.sqrt(((epsilon_x - epsilon_y) / 2)**2 + (gamma_xy / 2)**2)
  theta_mohr = np.linspace(0, 2 * np.pi, 200)
  mohr_circle_x = center + radius * np.cos(theta_mohr)
  mohr_circle_y = radius * np.sin(theta_mohr)

  # Convert strains to microstrain for display
  epsilon_x_prime_micro = epsilon_x_prime * 1e6
  epsilon_y_prime_micro = epsilon_y_prime * 1e6
  gamma_xy_prime_micro = gamma_xy_prime * 1e6

  # Plot the strain element
  fig, axes = plt.subplots(1, 2, figsize=(14, 7))

  # Plot the rotated square
  axes[0].plot(
    rotated_square[:, 0],
    rotated_square[:, 1],
    color="tab:blue",
    ls="-",
    lw=2,
    label="Strain Element",
  )

  # Plot normal strain vectors
  axes[0].quiver(
    right_normal_start[0],
    right_normal_start[1],
    right_normal_vector_rotated[0],
    right_normal_vector_rotated[1],
    angles="xy",
    scale_units="xy",
    scale=1,
    color="r",
    label="Right Normal Strain",
  )
  axes[0].quiver(
    top_normal_start[0],
    top_normal_start[1],
    top_normal_vector_rotated[0],
    top_normal_vector_rotated[1],
    angles="xy",
    scale_units="xy",
    scale=1,
    color="tab:orange",
    label="Top Normal Strain",
  )

  # Plot shear strain vectors
  axes[0].quiver(
    right_shear_start[0],
    right_shear_start[1],
    right_shear_vector_rotated[0],
    right_shear_vector_rotated[1],
    angles="xy",
    scale_units="xy",
    scale=1,
    color="g",
    label="Right Shear Strain",
  )
  axes[0].quiver(
    top_shear_start[0],
    top_shear_start[1],
    top_shear_vector_rotated[0],
    top_shear_vector_rotated[1],
    angles="xy",
    scale_units="xy",
    scale=1,
    color="tab:green",
    label="Top Shear Strain",
  )

  # Annotate strain arrows (in microstrain)
  if epsilon_x_prime >= 0:
    axes[0].text(
      right_normal_end[0] + 0.1,
      right_normal_end[1] + 0.1,
      f"εx': {abs(epsilon_x_prime_micro):.1f}",
      color="k",
      fontsize=12,
    )
  else:
    axes[0].text(
      right_normal_start[0] + 0.1,
      right_normal_start[1] + 0.1,
      f"εx': {abs(epsilon_x_prime_micro):.1f}",
      color="k",
      fontsize=12,
    )

  if epsilon_y_prime >= 0:
    axes[0].text(
      top_normal_end[0] + 0.1,
      top_normal_end[1] + 0.1,
      f"εy': {abs(epsilon_y_prime_micro):.1f}",
      color="k",
      fontsize=12,
    )
  else:
    axes[0].text(
      top_normal_start[0] + 0.1,
      top_normal_start[1] + 0.1,
      f"εy': {abs(epsilon_y_prime_micro):.1f}",
      color="k",
      fontsize=12,
    )

  axes[0].text(
    right_shear_end[0] + 0.1,
    right_shear_end[1] + 0.1,
    f"γx'y': {np.abs(gamma_xy_prime_micro):.1f}",
    color="k",
    fontsize=12,
  )
  axes[0].text(
    top_shear_end[0] + 0.1,
    top_shear_end[1] + 0.1,
    f"γx'y': {np.abs(gamma_xy_prime_micro):.1f}",
    color="k",
    fontsize=12,
  )

  axes[0].set_xlim(-4, 4)
  axes[0].set_ylim(-4, 4)
  axes[0].axhline(0, color="gray", lw=0.5, linestyle="--")
  axes[0].axvline(0, color="gray", lw=0.5, linestyle="--")
  axes[0].set_title("Strain Element")
  axes[0].set_aspect("equal", adjustable="box")
  axes[0].legend(loc="upper right")

  # Plot the Mohr Circle (convert to microstrain for display)
  mohr_circle_x_micro = mohr_circle_x * 1e6
  mohr_circle_y_micro = mohr_circle_y * 1e6

  axes[1].plot(mohr_circle_x_micro,
               mohr_circle_y_micro,
               color="tab:blue",
               ls="-",
               label="_nolegend_")
  axes[1].scatter([epsilon_x_prime_micro], [gamma_xy_prime_micro / 2],
                  color="r",
                  label="Right Strain Point")
  axes[1].scatter([epsilon_y_prime_micro], [-gamma_xy_prime_micro / 2],
                  color="tab:orange",
                  label="Top Strain Point")

  # Add the center point to Mohr Circle (in microstrain)
  center_micro = center * 1e6
  axes[1].scatter([center_micro], [0], color="black", label="_nolegend_")
  axes[1].text(center_micro + 20,
               20,
               f"({center_micro:.1f}, 0)",
               color="black",
               fontsize=10)

  # Connect strain points
  axes[1].plot(
      [epsilon_x_prime_micro, epsilon_y_prime_micro],
      [gamma_xy_prime_micro / 2, -gamma_xy_prime_micro / 2],
    color="gray",
    linestyle="--",
  )

  # Annotate strain points (in microstrain)
  axes[1].text(
    epsilon_x_prime_micro + 20,
    gamma_xy_prime_micro / 2 + 20,
    f"({epsilon_x_prime_micro:.1f}, {gamma_xy_prime_micro/2:.1f})",
    color="r",
    fontsize=10,
  )
  axes[1].text(
    epsilon_y_prime_micro + 20,
    -gamma_xy_prime_micro / 2 + 20,
    f"({epsilon_y_prime_micro:.1f}, {-gamma_xy_prime_micro/2:.1f})",
    color="tab:orange",
    fontsize=10,
  )

  # Annotate the rotation angle
  axes[1].text(
    epsilon_x_prime_micro + 20,
    gamma_xy_prime_micro / 2 - 16,
    f"Angle: {2 * angle:.1f}°",
    color="tab:blue",
    fontsize=12,
  )

  axes[1].axhline(0, color="gray", lw=0.5, linestyle="--")
  axes[1].axvline(0, color="gray", lw=0.5, linestyle="--")
  axes[1].set_xlabel(r"Normal Strain $\epsilon$ $(10^{-6})$")
  axes[1].set_ylabel(r"Shear Strain $\frac{\gamma}{2}$ $(10^{-6})$")

  # Adjust plot limits for microstrain
  radius_micro = radius * 1e6
  axes[1].set_xlim(center_micro - radius_micro - 100,
                   center_micro + radius_micro + 100)
  axes[1].set_ylim(radius_micro + 100,
                   -radius_micro - 100)  # Reverse the y-axis
  axes[1].set_title("Mohr's Circle for Strain")
  axes[1].legend()
  axes[1].set_aspect("equal", adjustable="box")

  plt.tight_layout()
  plt.show()


# Interactive sliders for user input (using microstrain ranges)
angle_slider = FloatSlider(value=0,
                           min=-180,
                           max=180,
                           step=0.5,
                           description="Angle:")
epsilon_x_slider = FloatSlider(value=200,
                               min=-1000,
                               max=1000,
                               step=10,
                               description=r"εx (10^-6):")
epsilon_y_slider = FloatSlider(value=-200,
                               min=-1000,
                               max=1000,
                               step=10,
                               description=r"εy (10^-6):")
gamma_xy_slider = FloatSlider(value=-300,
                              min=-1000,
                              max=1000,
                              step=10,
                              description=r"γxy (10^-6):")

interact(
  plot_strain_and_mohr_circle,
  angle=angle_slider,
  epsilon_x=epsilon_x_slider,
  epsilon_y=epsilon_y_slider,
  gamma_xy=gamma_xy_slider,
)

interactive(children=(FloatSlider(value=0.0, description='Angle:', max=180.0, min=-180.0, step=0.5), FloatSlid…

<function __main__.plot_strain_and_mohr_circle(angle, epsilon_x, epsilon_y, gamma_xy)>